In [ ]:
import pandas as pd
import os
import requests
import io
import matplotlib.pyplot as plt
import numpy as np
import nltk
nltk.download('punkt')
import operator
import re
import seaborn as sns

from wordcloud import WordCloud

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
path ='https://raw.githubusercontent.com/tanvi-ap/ClimateChange_StanceDetection/main/Climate_change_train.csv'
r = requests.get(path)
train_df = pd.read_csv(io.StringIO(r.text),index_col=0)

train_df.sample(5)

,Target,Tweet,Stance
ID,,,
10276,Climate Change is a Real Concern,Counting down the days until my PhD starts #77...,NONE
732,Climate Change is a Real Concern,Oh my lord! So hot and humid for few days now....,FAVOR
10341,Climate Change is a Real Concern,"#ClimateChangeDefinitions ""settled science"" = ...",AGAINST
623,Climate Change is a Real Concern,The Climate Change people are disgusting assho...,AGAINST
671,Climate Change is a Real Concern,Dear #seattleweather please stay exactly how y...,NONE


In [ ]:
train_df['Stance'].value_counts()

FAVOR      335
NONE       203
AGAINST     26
Name: Stance, dtype: int64

In [ ]:
x = train_df['Tweet']
y = train_df['Stance']

In [ ]:
# !pip install tweet-preprocessor
# import preprocessor as p

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words=stopwords.words('english')
stemmer=PorterStemmer()

In [ ]:
import re
cleaned_data=[]
for i in range(len(x)):
   tweet=re.sub('[^a-zA-Z]',' ',x.iloc[i])
   tweet=tweet.lower().split()
   tweet=[stemmer.stem(word) for word in tweet if (word not in stop_words)]
   tweet=' '.join(tweet)
   cleaned_data.append(tweet)

In [ ]:
print(cleaned_data)

['close door session begin decid entiti gcf semst', 'energiewend agenda de carbon camp long toler lie semst', 'climaspher stocker fish catch potenti could drop much area due oceanacidif cfcc semst', 'longer memori sterner sens justic w berri semst', 'boni futur sea level rise uniform differ coast ccfc journey semst', 'climaspher boni extrem weather event heat wave drought flood windstorm made frequent intens cfcc semst', 'camil parmesan speci chang habitat h cfcc semst', 'meta analys show speci chang live chang live camil parmesan cfcc semst', 'literarli took minut get bu old ladi go front everyon semst', 'grubb state energi shift us eu net produc net import emiss ccfc journey semst', 'interest speaker maarten van aalst feder humanitarian experi help climat scientist ccfc semst', 'put neck regard dion peac gm democraci transpar futur semst', 'grow food lawn lawn ecocid yard patriarchi ecocid speci dominoeffect semst', 'halcyon summer climat move doug kelbaugh cnu declarationsigne semst

In [ ]:
len(cleaned_data)

564

In [ ]:
sentiment_ordering = ['AGAINST', 'NONE', 'FAVOR']
y = y.apply(lambda x: sentiment_ordering.index(x))

In [ ]:
y

ID
10221    1
10222    2
10223    2
10224    1
10225    2
        ..
1004     2
1005     2
1006     2
1007     2
1008     2
Name: Stance, Length: 564, dtype: int64

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000)
X_fin=cv.fit_transform(cleaned_data).toarray()

In [ ]:
X_fin.shape

(564, 2516)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_fin,y,test_size=0.3)
model.fit(X_train,y_train)

MultinomialNB()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
cf=classification_report(y_test,y_pred)
print(cf)

              precision    recall  f1-score   support

           0       1.00      0.17      0.29         6
           1       0.74      0.59      0.66        59
           2       0.76      0.89      0.82       105

    accuracy                           0.76       170
   macro avg       0.84      0.55      0.59       170
weighted avg       0.76      0.76      0.75       170

